## JTSS APT setup

### This notebook details how to set up the JTSS observations in the APT, using the custom hMPT code to override shutters

In [232]:
import numpy as np
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord
import pandas as pd

import msa_planner
import importlib
importlib.reload(msa_planner)

<module 'msa_planner' from '/home/smmccrty/Research/JWST/hMPT/msa_planner.py'>

In [234]:
# read in catalogs, standardize column names, and stack. Ignoring flux columns here.
JTSS_catalog_GN = Table.read('jtss_data/JTSS_catalog_GN_v4.fits')['ID','RA','DEC','z_a','dh_select','sub','F444W_max']
mask = (JTSS_catalog_GN['dh_select']>=1)&((JTSS_catalog_GN['z_a']>2)|(JTSS_catalog_GN['sub']<1.0/4.0)|(JTSS_catalog_GN['F444W_max']>57.54))
JTSS_catalog_GN = JTSS_catalog_GN[mask]['ID','RA','DEC','z_a','dh_select']
JTSS_catalog_GN['ID'] = JTSS_catalog_GN['ID'].astype(int)

JTSS_catalog_GS = Table.read('jtss_data/JTSS_catalog_GS_v4.fits')['ID','RA','DEC','z_a','dh_select','sub','F444W_max']
mask = (JTSS_catalog_GS['dh_select']>=1)&((JTSS_catalog_GS['z_a']>2)|(JTSS_catalog_GS['sub']<1.0/4.0)|(JTSS_catalog_GS['F444W_max']>57.54))
JTSS_catalog_GS = JTSS_catalog_GS[mask]['ID','RA','DEC','z_a','dh_select']
JTSS_catalog_GS['ID'] = JTSS_catalog_GS['ID'].astype(int)

JTSS_catalog_UNCOVER = Table.read('jtss_data/JTSS_catalog_UNCOVER_v4.fits')['id','ra','dec','z_phot','dh_select','sub','f_f444w']
mask = (JTSS_catalog_UNCOVER['dh_select']>=1)&((JTSS_catalog_UNCOVER['z_phot']>2)|(JTSS_catalog_UNCOVER['sub']<1.0/4.0)|(JTSS_catalog_UNCOVER['f_f444w']>57.54))
JTSS_catalog_UNCOVER = JTSS_catalog_UNCOVER[mask]['id','ra','dec','z_phot','dh_select']
JTSS_catalog_UNCOVER.rename_columns(JTSS_catalog_UNCOVER.colnames, JTSS_catalog_GN.colnames)  # ensure same column names
JTSS_catalog_UNCOVER['ID'] = JTSS_catalog_UNCOVER['ID'].astype(int)

In [235]:
# read in pointings
final_pointings = np.zeros((7,4,3,3)) # 7 tiles, 4 pointings per tile, 3 dithers per pointing, 3 values (RA, Dec, V3PA)

with open('jtss_data/pointings', 'r') as f:
    lines = f.readlines()
    current_tile = 0
    current_pointing = 0
    current_dither = 0
    for i, line in enumerate(lines):
        if line == '\n':
            current_tile += 1
            current_pointing = 0
            current_dither = 0
            continue
        else:
            parts = line.split()
            final_pointings[current_tile, current_pointing, current_dither] = [float(parts[1]), float(parts[2]), float(parts[3])]
            current_dither += 1
            if current_dither == 3:
                current_pointing += 1
                current_dither = 0

##### Save the standardized catalogs, calculate EoE correction, print out pointings for entry into APT

In [236]:
# Need to add '#' to beginning of header line for APT
with open(f'jtss_data/GOODS_catalog.csv', 'w') as f:
    f.write('# ID,RA,DEC,Redshift,Number\n')  # APT recognizes 'Number' instead of dh_select and Redshift instead of z_a
    JTSS_catalog_GS.to_pandas().to_csv(f, index=False, header=False)

with open(f'jtss_data/GOODN_catalog.csv', 'w') as f:
    f.write('# ID,RA,DEC,Redshift,Number\n')  # APT recognizes 'Number' instead of dh_select and Redshift instead of z_a
    JTSS_catalog_GN.to_pandas().to_csv(f, index=False, header=False)

with open(f'jtss_data/UNCOVER_catalog.csv', 'w') as f:
    f.write('# ID,RA,DEC,Redshift,Number\n')  # APT recognizes 'Number' instead of dh_select and Redshift instead of z_a
    JTSS_catalog_UNCOVER.to_pandas().to_csv(f, index=False, header=False)

In [208]:
def EoE_correction(pointing, catalog):
    ra_c = np.median(catalog['RA']) * np.pi/180
    dec_c = np.median(catalog['DEC']) * np.pi/180
    ra_p = pointing[0] * np.pi/180
    dec_p = pointing[1] * np.pi/180
    v3pa = pointing[2]

    num = np.sin(ra_p - ra_c) * (np.sin(dec_c) + np.sin(dec_p))
    denom = np.cos(dec_c) * np.cos(dec_p) + np.cos(ra_p - ra_c) * (1 + np.sin(dec_c) * np.sin(dec_p))

    d_theta = np.arctan(num/denom) * 180/np.pi

    return d_theta

In [237]:
# print everything out for copying into APT
tile_catalogs = ['GOODS','GOODS','GOODS','GOODS','GOODN','GOODN','UNCOVER']
for t,tile in enumerate(final_pointings):
    catalog = pd.read_csv(f'jtss_data/{tile_catalogs[t]}_catalog.csv', comment='#', names=['ID','RA','DEC','Redshift','Number'], index_col=False)
    print(f"Coords for tile {t}:")
    for p,pointing in enumerate(tile):
        print(f'Pointing {p}')
        for d,dither in enumerate(pointing):
            d_theta = EoE_correction(dither, catalog)
            APA = (dither[2]+180-41.42543)%360
            print(f'Dither {d}: RA={dither[0]}, Dec={dither[1]}, APA={APA}, APA for MPT={APA-d_theta}')

    print('\n')

Coords for tile 0:
Pointing 0
Dither 0: RA=52.9753458, Dec=-27.806397, APA=78.57457, APA for MPT=78.52710334107853
Dither 1: RA=52.97560197, Dec=-27.80652836, APA=78.57457, APA for MPT=78.527222754353
Dither 2: RA=52.97508964, Dec=-27.80626564, APA=78.57457, APA for MPT=78.52698393298864
Pointing 1
Dither 0: RA=52.99972685, Dec=-27.80203897, APA=78.57457, APA for MPT=78.5384809256446
Dither 1: RA=52.99998301, Dec=-27.80217032, APA=78.57457, APA for MPT=78.53860035033163
Dither 2: RA=52.9994707, Dec=-27.80190761, APA=78.57457, APA for MPT=78.53836150614784
Pointing 2
Dither 0: RA=52.98027315, Dec=-27.82796103, APA=80.57457, APA for MPT=80.52938608380792
Dither 1: RA=52.98052402, Dec=-27.82810022, APA=80.57457, APA for MPT=80.52950306526883
Dither 2: RA=52.98002227, Dec=-27.82782185, APA=80.57457, APA for MPT=80.52926909821107
Pointing 3
Dither 0: RA=53.0046542, Dec=-27.823603, APA=80.57457, APA for MPT=80.54076756001741
Dither 1: RA=53.00490506, Dec=-27.82374219, APA=80.57457, APA for M

#### Now, some button clicking in APT

- Targets -> Import MSA source catalog -> GOODS_catalog.csv
- Tile 0-3 GOODS, 4-5 GOODN, 6 UNCOVER
- Click GOODS_catalog.csv on the left -> New Candidate Set
- Name = GOODS_faint -> Add filter -> New number column filter -> min and max = 2, this selects Number (dh_select)=2 faint sources
- No need to create bright candidate set because these will include all sources
- Repeat for the three catalogs

- Now, -> MSA Planning Tool
- MSA catalogs = GOODS_catalog, Primary Candidate List = GOODS_faint, Filler Candidate List = None
- Enter APA for the first pointing (corrected for EoE)
- 1 Shutter Slitler, Unconstrained
- Fixed pointings, enter three dithers
- Grating/Filters, select G395M/F290LP and G235M/F170LP
- Multiple sources per row, min seperation 4
- Allow sources in areas affected by stuck open shutters
- Plan name = tXpX_faint
- Generate plan
- Repeat for bright but Primary Candidate List is the whole catalog and use G395H/F290LP and G235H/F170LP, plan name = tXpX_bright

- Select "Plan"
- Ctrl select both bright and faint plans -> Create Observation
- Order them as desired
- APA can only be set by MPT, so we can reduce the number of clicks in APT by using duplication but only at the level of MSAs with the same APA. 
- In the current formulation, the APA supplied to APT depends on position through the EoE correction, so in principle nothing has the same APA, but we can treat each dither as having the same APA without much loss of accuracy
- You can cut down clicks based on how much inaccuracy we are willing to take on, i.e. if we are fine treating the whole tile as a single APA we could just make one pointing and duplicate 4 times
- For now I will just assume same APA on dither level

- Make new observation folder between files

### Now creating custom MSA config files to import into APT
- We need to correct for the DVA, which requires knowing the theta parameter. At the moment, the only way to do this is to set up the observation in the APT as above and then File->Export->MSA target info-> search the xml file for theta. This can be skipped if theta is known. Also may not even matter at this stage.
- I have also downloaded the MSA Target Info files and a few of the MSA configuration files generated by MPT from the above in order to check consistency before overriding with our own config files

In [253]:
# Thetas that I have already retrieved, could prob just take one per tile because the correction varies slowly with theta
# 1 per pointing, 4 per tile
thetas = [69.26422998504829, 69.27076990195069, 70.40915890612285, 70.41562939432862, 
          69.2819210520432, 69.2884657004088, 70.42666892243233, 70.43314429880422, 
          102.40455032065915, 102.40818935683183, 103.45897277119036, 103.46264341756748, 
          102.41567358205592, 102.41931246131986, 103.47020976729384, 103.47388023402706, 
          105.27846798018942, 105.26995679350209, 104.37846414389841, 104.36992677400227,
          105.25293328272122, 105.24442122837142, 104.35285449920607, 104.3443196008364,
          149.8499549276264, 149.8484558282443, 150.36681607317126, 150.3658619053453]

In [254]:
attempt_dir = 'JTSS_v1'

In [260]:
import glob

total_eff_exp = 0
total_spectra = 0

# get MSA models for the total catalog
catalogs = []
for cat in ['GOODS', 'GOODN', 'UNCOVER']:
    catalogs.append(pd.read_csv(f'jtss_data/{cat}_catalog.csv', comment='#', names=['ID','RA','DEC','Redshift','Number'], index_col=False))
master_catalog = pd.concat(catalogs)
faint_mask = master_catalog['Number'] == 2
MSAModel_faint = msa_planner.MSAModel(shutter_mask_file='esa_msa_map_APT_2025.5.3_v2.dat',
                                distortion_file='msa_v2v3.dat',
                                ra_sources = master_catalog['RA'][faint_mask].values,
                                dec_sources = master_catalog['DEC'][faint_mask].values,
                                buffer=msa_planner.UNCONSTRAINED)
MSAModel_bright = msa_planner.MSAModel(shutter_mask_file='esa_msa_map_APT_2025.5.3_v2.dat',
                                distortion_file='msa_v2v3.dat',
                                ra_sources = master_catalog['RA'].values,
                                dec_sources = master_catalog['DEC'].values,
                                buffer=msa_planner.UNCONSTRAINED)

for t, tile in enumerate(final_pointings):
    
    eff_exp_faint = np.zeros(len(master_catalog[faint_mask]))
    spectra_faint = np.zeros(len(master_catalog[faint_mask]))

    for p, pointing in enumerate(tile):
        for d, dither in enumerate(pointing):
            for type in ['faint','bright']:
                if type == 'faint':
                    MSAModel = MSAModel_faint
                else:
                    MSAModel = MSAModel_bright

                ra_msa, dec_msa, ang_v3 = dither
                theta = thetas[t+p]
                print(f'Tile {t}, Pointing {p}, Dither {d}, Type {type}')

                axy = msa_planner.radec_to_Axy(MSAModel.ra_sources, MSAModel.dec_sources, ra_msa, dec_msa, ang_v3, theta=theta)
                shutters = msa_planner.find_shutter_from_Axy(axy, MSAModel.spline_models)
                available = MSAModel.apply_shutter_mask(shutters, MSAModel.shutter_mask)
                centered = MSAModel.apply_shutter_centration(shutters, MSAModel.buffer)
                shut_mask = available & centered
                if type=='faint':
                    use = np.where(available&centered,1,0)
                    throughput = MSAModel.estimate_slit_flux(shutters) * use
                    eff_exp_faint += throughput
                    spectra_faint += use
                else:
                    use = np.where(available&centered,1,0)
                    throughput = MSAModel.estimate_slit_flux(shutters) * use
                    eff_exp_faint += throughput[faint_mask]
                    spectra_faint += use[faint_mask]

                output_csv = f'jtss_data/{attempt_dir}/t{t}p{p}d{d}_{type}_msaconfig.csv'
                msa_planner.make_msa_config(np.array(shutters)[:,shut_mask], 'esa_msa_map_APT_2025.5.3_v2.dat', output_csv)

                # now comparing to MPT output
                mpt_output_file = glob.glob(f'jtss_data/{attempt_dir}/obs{t*4+p+1}*-exp*-c{d+1} : t{t}p{p}_{type}e*.csv')
                if len(mpt_output_file) == 0:
                    print(f'No MPT output file found for t{t}p{p}d{d} {type}')
                    continue
                msa_planner.check_model(mpt_output_file[0], dither, MSAModel.spline_models, theta, plot=False)

    print('\n')
    print("------------------------------------")
    print(f"Tile {t} stats for faint objects:")
    print(f"{np.sum(spectra_faint)} spectra of {np.sum(spectra_faint>0)} unique sources")
    print(f"Mean eff exp: {np.mean(eff_exp_faint[spectra_faint>0])}")
    print(f"Mean number of spectra: {np.mean(spectra_faint[spectra_faint>0])}")
    print("------------------------------------")
    print('\n')

    total_eff_exp += eff_exp_faint
    total_spectra += spectra_faint

print('\n')
print("================================")
print("Total stats:")
print(f"{np.sum(total_spectra)} spectra of {np.sum(total_spectra>0)} unique sources")
print(f"Mean eff exp: {np.mean(total_eff_exp[total_spectra>0])}")
print(f"Mean number of spectra: {np.mean(total_spectra[total_spectra>0])}")

Tile 0, Pointing 0, Dither 0, Type faint
0 sources were marked out of bounds (quad<1)
0 quadrants are different between hMPT and MPT
0 sources were off by >0.5 in row
0 sources were off by >0.5 in col
Median row error by quad: 0.0174, 0.0187, 0.0138, 0.0147
Median col error by quad: -0.0078, 0.0001, 0.0061, 0.0064
Tile 0, Pointing 0, Dither 0, Type bright
0 sources were marked out of bounds (quad<1)
0 quadrants are different between hMPT and MPT
0 sources were off by >0.5 in row
0 sources were off by >0.5 in col
Median row error by quad: 0.0174, 0.0188, 0.0138, 0.0147
Median col error by quad: -0.0077, 0.0001, 0.0060, 0.0063
Tile 0, Pointing 0, Dither 1, Type faint
2 sources were marked out of bounds (quad<1)
0 quadrants are different between hMPT and MPT
0 sources were off by >0.5 in row
0 sources were off by >0.5 in col
Median row error by quad: 0.0177, 0.0190, 0.0136, 0.0145
Median col error by quad: -0.0083, 0.0006, 0.0057, 0.0070
Tile 0, Pointing 0, Dither 1, Type bright
2 sources

In [261]:
# simple ChatGPT function to compare the msa config files from MPT and from our code
from collections import Counter

def compare_config_files(file1, file2):
    df1 = pd.read_csv(file1, header=None, comment='#').astype(str)
    df2 = pd.read_csv(file2, header=None, comment='#').astype(str)

    # Make sure shapes match
    if df1.shape != df2.shape:
        raise ValueError("Files have different shapes!")

    print('number of sources in MPT:', (df1 == '0').sum().sum())
    print('number of sources in hMPT:', (df2 == '0').sum().sum())

    # Find boolean mask where file1 is '0', and file2 is NOT '0'
    mask = (df1 == '0') & (df2 != '0')

    # How many such cases?
    count = mask.sum().sum()
    print(f"Number of cells where MPT is '0' and hMPT is not '0': {count}")

    # Get a flat series of all corresponding file2 values
    mismatched_values = df2.to_numpy()[mask]

    # Count which numbers/letters those are:
    values_counter = Counter(mismatched_values)
    print("Counts of other values in hMPT:")
    for value, freq in values_counter.items():
        print(f"{value}: {freq}")

In [264]:
# I just downloaded the first MSA (p0d0_faint) from each tile to compare
for t, _ in enumerate(final_pointings):
    file1 = f'jtss_data/{attempt_dir}/t{t}p0d0_mptmsa_config.csv' # MPT output
    file2 = f'jtss_data/{attempt_dir}/t{t}p0d0_faint_msaconfig.csv' # our output
    compare_config_files(file1, file2)

number of sources in MPT: 1755
number of sources in hMPT: 3292
Number of cells where MPT is '0' and hMPT is not '0': 42
Counts of other values in hMPT:
1: 39
x: 3
number of sources in MPT: 1844
number of sources in hMPT: 3388
Number of cells where MPT is '0' and hMPT is not '0': 43
Counts of other values in hMPT:
1: 40
x: 3
number of sources in MPT: 1914
number of sources in hMPT: 3500
Number of cells where MPT is '0' and hMPT is not '0': 69
Counts of other values in hMPT:
1: 67
x: 2
number of sources in MPT: 1877
number of sources in hMPT: 3542
Number of cells where MPT is '0' and hMPT is not '0': 67
Counts of other values in hMPT:
1: 64
x: 3
number of sources in MPT: 1988
number of sources in hMPT: 3624
Number of cells where MPT is '0' and hMPT is not '0': 48
Counts of other values in hMPT:
1: 43
x: 5
number of sources in MPT: 1935
number of sources in hMPT: 3578
Number of cells where MPT is '0' and hMPT is not '0': 53
Counts of other values in hMPT:
1: 49
x: 4
number of sources in M

So we missed ~50 sources that MPT got, likely due to residual model errors, but overall have many more

### Now, getting the custom MSAs into APT

- They should easy to match to the correct line in the observation section, knowing that the dithers will be denoted c1,c2,c3 in APT
- Once one msa is imported for a gratings pair, it can be selected in the menu of the other grating line, and this should avoid an MSA change in the timeline between them

#### Observation section checklist (parallels, ordering, etc.)

- Check coordinated parallel
- MSATA
- Dither type None
- Exposures: 16 groups/int, 3 integrations, with NRSIRS2
- The MSA ordering should auto be this but to check:
    - Dither order 0 1 2 0 1 2
    - G235 first G395 second
- Confirmation images
    - The confirmation images should be “After Target ACQ and New MSA Config”
    - Then NRSIRS2RAPID readout with 3 groups, 1 integration, in each of the 6 rows.
- For NC section
    - Module: All
    - Subarray: Full
    - DEEP8 (not DEEP2), 6 groups, 3 integrations
    - F090W F444W dither 1 (the dither pointing doesn’t appear in APT, just writing so I can keep track)
    - F115W F410M dither 1
    - F115W F410M dither 2
    - F150W F356W dither 2
    - F150W F356W dither 3
    - F200W F277W dither 3
    - F200W F277W dither 1
    - F150W F356W dither 1
    - F200W F277W dither 2
    - F090W F444W dither 2
    - F090W F444W dither 3
    - F115W F410M dither 3
- Special Requirement, for the Aperture PA Range to
be the same as the Implicit Requirement.

## Total clicking time: ~an entire day :D